<a href="https://colab.research.google.com/github/ajayramaraju/Machine-Learning/blob/main/CNN_Regression_to_predict_porosity_of_given_iimage_(Base_Line_Model).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow



```
# This is formatted as code
```



In [ ]:
import os
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator


data_dir = '/content/drive/MyDrive/microstructure_dataset'


datagen = ImageDataGenerator(
    rotation_range=20,  # Randomly rotate images by up to 20 degrees
    #width_shift_range=0.2,
    shear_range=0.2,  # Shear transformations
    zoom_range=0.2,  # Randomly zoom in/out by up to 20%
    horizontal_flip=True,  # Randomly flip horizontally
    #fill_mode='constant'  # Fill mode for empty pixels
)

# Loop through our porosity directories and apply data augmentation
for porosity_dir in os.listdir(data_dir):
    if os.path.isdir(os.path.join(data_dir, porosity_dir)):
        print(f"Augmenting images in {porosity_dir}...")
        porosity_path = os.path.join(data_dir, porosity_dir)
        images = os.listdir(porosity_path)
        for image in images:
            img = keras.preprocessing.image.load_img(os.path.join(porosity_path, image))
            img = keras.preprocessing.image.img_to_array(img)
            img = img.reshape((1,) + img.shape)


            i = 0
            for batch in datagen.flow(img, save_to_dir=porosity_path, save_prefix='image', save_format='jpg'):
                i += 1
                if i == 4:
                    break

print("Data augmentation complete.")

In [ ]:
import os
import random
import shutil
from sklearn.model_selection import train_test_split

# Set the paths
data_dir = '/content/drive/MyDrive/microstructure_dataset'
output_dir = '/content/drive/MyDrive/project_directory'

# Define the split ratios (e.g., 80-10-10)
train_ratio = 0.80
validation_ratio = 0.1
test_ratio = 0.1

# Create directories for the three sets
os.makedirs(os.path.join(output_dir, 'training'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'validation'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'testing'), exist_ok=True)

# Loop through the porosity directories and split the data
for porosity_dir in os.listdir(data_dir):
    if os.path.isdir(os.path.join(data_dir, porosity_dir)):
        images = os.listdir(os.path.join(data_dir, porosity_dir))
        random.shuffle(images)

        # Shuffleing to randomize the order
        train_images, validation_images = train_test_split(images, test_size=validation_ratio, random_state=42)
        test_images = list(set(images) - set(train_images) - set(validation_images))

        for image in train_images:
            source = os.path.join(data_dir, porosity_dir, image)
            destination = os.path.join(output_dir, 'training', porosity_dir, image)
            os.makedirs(os.path.dirname(destination), exist_ok=True)
            shutil.copy(source, destination)

        for image in validation_images:
            source = os.path.join(data_dir, porosity_dir, image)
            destination = os.path.join(output_dir, 'validation', porosity_dir, image)
            os.makedirs(os.path.dirname(destination), exist_ok=True)
            shutil.copy(source, destination)

        for image in test_images:
            source = os.path.join(data_dir, porosity_dir, image)
            destination = os.path.join(output_dir, 'testing', porosity_dir, image)
            os.makedirs(os.path.dirname(destination), exist_ok=True)
            shutil.copy(source, destination)

print("Data splitting complete.")

Data splitting complete.


Data Pre Processing:

In [ ]:
import os
import cv2
import numpy as np

# Define the paths to your data
data_dir = '/content/drive/MyDrive/project_directory'
output_dir = '/content/drive/MyDrive/preprocessed_data'

# Create directories for preprocessed data
os.makedirs(os.path.join(output_dir, 'training'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'validation'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'testing'), exist_ok=True)

# Function to preprocess and save images
def preprocess_images(source_dir, destination_dir):
    for porosity_dir in os.listdir(source_dir):
        if os.path.isdir(os.path.join(source_dir, porosity_dir)):
            for image in os.listdir(os.path.join(source_dir, porosity_dir)):
                img_path = os.path.join(source_dir, porosity_dir, image)
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Convert to grayscale
                img = cv2.resize(img, (224, 224))  # Resize to a standard size (adjust as needed)
                img = img / 255.0  # Normalize pixel values to [0, 1]

                # Save the preprocessed image
                destination_path = os.path.join(destination_dir, porosity_dir, image)
                os.makedirs(os.path.dirname(destination_path), exist_ok=True)
                cv2.imwrite(destination_path, (img * 175).astype(np.uint8))  # Convert back to 8-bit format

# Preprocess the training set
preprocess_images(os.path.join(data_dir, 'training'), os.path.join(output_dir, 'training'))

# Preprocess the validation set
preprocess_images(os.path.join(data_dir, 'validation'), os.path.join(output_dir, 'validation'))

# Preprocess the testing set
preprocess_images(os.path.join(data_dir, 'testing'), os.path.join(output_dir, 'testing'))

print("Data preprocessing complete.")




Data preprocessing complete.


Model:

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

# Create a Sequential model
model = keras.Sequential()

# Convolutional layers
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(2, 2))
model.add(layers.Flatten())
model.add(layers.Dense(1,))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

Training:

In [ ]:
X_train = []
Y_train = []


for porosity_dir in os.listdir('/content/drive/MyDrive/preprocessed_data/training'):
    if os.path.isdir(os.path.join('/content/drive/MyDrive/preprocessed_data/training', porosity_dir)):
        # Load images in this porosity category
        image_paths = os.listdir(os.path.join('/content/drive/MyDrive/preprocessed_data/training', porosity_dir))
        for image_path in image_paths:
            # Load the image
            img = cv2.imread(os.path.join('/content/drive/MyDrive/preprocessed_data/training', porosity_dir, image_path), cv2.IMREAD_GRAYSCALE)
            # Append the image to X_train
            X_train.append(img)
            # Extract the porosity value from the directory name
            label = int(porosity_dir.replace('porosity_', ''))
            # Append the label to Y_train
            Y_train.append(label)

# Convert X_train and Y_train to NumPy arrays for compatibility with model.fit
X_train = np.array(X_train)
Y_train = np.array(Y_train)

# Verify shapes
print("X_train shape:", X_train.shape)
print("Y_train shape:", Y_train.shape)

print(X_train)
print(Y_train)





X_train shape: (194, 224, 224)
Y_train shape: (194,)
[[[175 175 175 ... 175 175 175]
  [175 175 175 ... 175 175 175]
  [175 175 175 ... 175 175 175]
  ...
  [175 175 175 ... 175 175 175]
  [175 175 175 ... 175 175 175]
  [175 175 175 ... 175 175 175]]

 [[175 175 175 ... 175 175 175]
  [175 175 175 ... 175 175 175]
  [175 175 175 ... 175 175 175]
  ...
  [175 175 175 ... 175 175 175]
  [175 175 175 ... 175 175 175]
  [175 175 175 ... 175 175 175]]

 [[175 175 175 ... 175 175 175]
  [175 175 175 ... 175 175 175]
  [175 175 175 ... 175 175 175]
  ...
  [175 175 175 ... 175 175 175]
  [175 175 175 ... 175 175 175]
  [175 175 175 ... 175 175 175]]

 ...

 [[175 175 175 ... 175 175 175]
  [175 175 175 ... 175 175 175]
  [175 175 175 ... 175 175 175]
  ...
  [175 175 175 ... 175 175 175]
  [175 175 175 ... 175 175 175]
  [175 175 175 ... 175 175 175]]

 [[175 175 175 ... 175 175 175]
  [175 175 175 ... 175 175 175]
  [175 175 175 ... 175 175 175]
  ...
  [175 175 175 ... 175 175 175]
  [175 

Training


In [ ]:
epochs = 10

# Train the model
history = model.fit(X_train, Y_train, epochs=epochs, validation_split=0.1)

Epoch 1/10
6/6 [==============================] - 21s 3s/step - loss: 773829.3125 - mae: 464.9102 - mse: 773829.3125 - val_loss: 227.2538 - val_mae: 13.8271 - val_mse: 227.2538
Epoch 2/10
6/6 [==============================] - 20s 3s/step - loss: 318.5288 - mae: 14.5270 - mse: 318.5288 - val_loss: 65.8403 - val_mae: 6.8465 - val_mse: 65.8403
Epoch 3/10
6/6 [==============================] - 18s 3s/step - loss: 123.3511 - mae: 9.4386 - mse: 123.3511 - val_loss: 121.9852 - val_mae: 9.7131 - val_mse: 121.9852
Epoch 4/10
6/6 [==============================] - 18s 3s/step - loss: 97.1469 - mae: 8.3420 - mse: 97.1469 - val_loss: 107.4259 - val_mae: 9.2096 - val_mse: 107.4259
Epoch 5/10
6/6 [==============================] - 18s 3s/step - loss: 85.7441 - mae: 7.7906 - mse: 85.7441 - val_loss: 34.8374 - val_mae: 4.9234 - val_mse: 34.8374
Epoch 6/10
6/6 [==============================] - 20s 3s/step - loss: 97.5715 - mae: 8.1286 - mse: 97.5715 - val_loss: 30.3188 - val_mae: 4.5574 - val_mse: 30

In [ ]:
import os
import cv2
import numpy as np


test_data_dir = '/content/drive/MyDrive/preprocessed_data/testing'

# Check if the test data directory exists
if not os.path.exists(test_data_dir):
    print(f"Test data directory not found: {test_data_dir}")
else:
    # Create empty lists to store test data
    X_test = []
    Y_test = []

    # Iterate through the test data directories
    for porosity_dir in os.listdir(test_data_dir):
        if os.path.isdir(os.path.join(test_data_dir, porosity_dir)):
            # Load images in this porosity category
            image_paths = os.listdir(os.path.join(test_data_dir, porosity_dir))
            for image_path in image_paths:
                # Load the test image
                img = cv2.imread(os.path.join(test_data_dir, porosity_dir, image_path), cv2.IMREAD_GRAYSCALE)

                # Append the test image to X_test
                X_test.append(img)

                # Extract the true porosity value from the directory name
                label = int(porosity_dir.replace('porosity_', ''))

                # Append the true porosity value to Y_test
                Y_test.append(label)

    # Convert X_test and Y_test to NumPy arrays
    X_test = np.array(X_test)
    Y_test = np.array(Y_test)

    # Verify shapes
    print("X_test shape:", X_test.shape)
    print("Y_test shape:", Y_test.shape)

X_test shape: (41, 224, 224)
Y_test shape: (41,)


In [ ]:
loss, mae, mse = model.evaluate(X_test, Y_test)

# Print the evaluation results
print("Test Loss (MSE):", loss)
print("Test MAE:", mae)
print("Test MSE:", mse)

2/2 [==============================] - 1s 306ms/step - loss: 17.1860 - mae: 3.2502 - mse: 17.1860
Test Loss (MSE): 17.186031341552734
Test MAE: 3.2501883506774902
Test MSE: 17.186031341552734


In [ ]:
import cv2
import numpy as np

image_path = '/content/drive/MyDrive/preprocessed_data/testing/porosity_40/40 (3).jpg'  # Provide the path to your image
input_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

# Preprocess the image
input_image = cv2.resize(input_image, (224, 224))
input_image = input_image / 255.0  # Normalize pixel values to [0, 1]
input_image = input_image.reshape(1, 224, 224, 1)  # Reshape for model input
# Use the model for prediction
predicted_porosity = model.predict(input_image)

# Print the predicted porosity
print("Predicted Porosity:", predicted_porosity[0][0])

1/1 [==============================] - 0s 77ms/step
Predicted Porosity: 0.18628031
